In [0]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import ShuffleSplit, KFold

# train = pd.read_csv("C:/Users/swl3guest/CE4032/datasets/train_modified_v2.csv")
# test = pd.read_csv("C:/Users/swl3guest/CE4032/datasets/test_modified_v2.csv")


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
train_df = pd.read_csv("/content/drive/My Drive/CE4032/dataset/modified_train_v3.csv")
test_df = pd.read_csv("/content/drive/My Drive/CE4032/dataset/modified_test_v3.csv")
ids = test_df['TRIP_ID']
train_df.head()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DATE,END_TIME,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C,DURATION
0,1372636858620000589,NaN,NaN,20000589,1372636858,2013-07-01 01:00:58+01:00,1372637188,0,8,-8.618643,41.141412,0.149319,1830.976883,19,41.154507,-8.630829,1341.015903,2.185836,1780.005072,-0.611154,2623.280294,11.041239,12.725186,0,0,1,1,0,0,330
1,1372636858620000589,NaN,NaN,20000589,1372636858,2013-07-01 01:00:58+01:00,1372637188,0,8,-8.618643,41.141412,0.149319,1830.976883,4,41.144373,-8.623953,1645.635018,2.185903,553.896759,-0.933391,422.296381,13.232000,14.073105,0,0,1,1,0,0,330
2,1372637303620000596,NaN,7.0,20000596,1372637303,2013-07-01 01:08:23+01:00,1372637573,0,8,-8.639847,41.159826,1.686860,2063.429934,7,41.162580,-8.656434,3482.655006,2.186018,1423.528654,-1.353639,1143.447103,13.561319,18.148117,0,1,0,1,0,0,270
3,1372636951620000320,NaN,NaN,20000320,1372636951,2013-07-01 01:02:31+01:00,1372637911,0,8,-8.612964,41.140359,-0.105214,1938.539952,11,41.145174,-8.630082,1858.247962,2.185963,1531.865239,-1.213231,1368.236297,9.194587,13.289274,0,0,1,1,0,0,960
4,1372636951620000320,NaN,NaN,20000320,1372636951,2013-07-01 01:02:31+01:00,1372637911,0,8,-8.612964,41.140359,-0.105214,1938.539952,17,41.152464,-8.628579,1248.274734,2.185839,1878.613544,-0.770793,2534.477061,12.578108,11.892797,0,0,1,1,0,0,960


In [4]:
train_df['DURATION'] = train_df['DURATION'] / 15
y= np.log(train_df['DURATION']) + 1
train_cl = train_df.drop(columns=['TRIP_ID','TIMESTAMP','DATE', 'END_TIME',  'DURATION', 'ORIGIN_CALL',	'ORIGIN_STAND'])
test_cl= test_df.drop(columns=[ 'TRIP_ID','TIMESTAMP','DATE', 'END_TIME', 'ORIGIN_CALL',	'ORIGIN_STAND'])
train_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000589,0,8,-8.618643,41.141412,0.149319,1830.976883,19,41.154507,-8.630829,1341.015903,2.185836,1780.005072,-0.611154,2623.280294,11.041239,12.725186,0,0,1,1,0,0
1,20000589,0,8,-8.618643,41.141412,0.149319,1830.976883,4,41.144373,-8.623953,1645.635018,2.185903,553.896759,-0.933391,422.296381,13.232000,14.073105,0,0,1,1,0,0
2,20000596,0,8,-8.639847,41.159826,1.686860,2063.429934,7,41.162580,-8.656434,3482.655006,2.186018,1423.528654,-1.353639,1143.447103,13.561319,18.148117,0,1,0,1,0,0
3,20000320,0,8,-8.612964,41.140359,-0.105214,1938.539952,11,41.145174,-8.630082,1858.247962,2.185963,1531.865239,-1.213231,1368.236297,9.194587,13.289274,0,0,1,1,0,0
4,20000320,0,8,-8.612964,41.140359,-0.105214,1938.539952,17,41.152464,-8.628579,1248.274734,2.185839,1878.613544,-0.770793,2534.477061,12.578108,11.892797,0,0,1,1,0,0


In [5]:
test_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,CUT_OFF_DIST_FROM_CC,HEADER_CUT_OFF_TO_CC,CUT_OFF_DIST_FROM_ORIGIN,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000542,3,18,-8.585676,41.148522,-1.182260,2691.258243,11,41.146623,-8.584884,2837.910054,2.019103,221.575614,2.837288,513.623887,2.411640,10.243918,0,1,0,0,0,1
1,20000108,3,18,-8.610876,41.145570,-0.273847,1399.904997,40,41.163597,-8.601894,1309.291868,1.043254,2143.324472,0.358853,2282.133126,3.340840,5.146115,0,1,0,0,0,1
2,20000370,3,18,-8.585739,41.148558,-1.182897,2684.853296,40,41.167719,-8.574903,3572.711105,1.252324,2318.288447,0.402469,3911.607406,5.660043,6.645085,0,1,0,0,0,1
3,20000492,3,18,-8.613963,41.141169,-0.065131,1841.553918,8,41.140980,-8.614638,1859.765020,3.107550,60.372488,-1.926755,176.758074,1.064723,2.607521,0,1,0,0,0,1
4,20000621,3,18,-8.619903,41.148036,0.338632,1137.845589,2,41.148036,-8.619894,1137.595194,-2.803534,0.754424,1.570796,0.754424,0.050295,0.050295,0,1,0,0,0,1


In [0]:
train = train_cl

In [0]:
test = test_cl
feature_names = list(train.columns)

In [0]:
X = np.array(train, dtype=np.float)
X_test = np.array(test, dtype=np.float)

In [21]:
clf = GradientBoostingRegressor(n_estimators=10, learning_rate = 0.15, max_depth=3, max_features = 15, min_weight_fraction_leaf = 0.15, random_state=21, verbose = 1)
clf.fit(X, y)

      Iter       Train Loss   Remaining Time 
         1           0.3143            1.83m
         2           0.2898            1.52m
         3           0.2704            1.28m
         4           0.2588            1.09m
         5           0.2469           54.08s
         6           0.2377           44.02s
         7           0.2303           33.04s
         8           0.2246           22.16s
         9           0.2209           11.11s
        10           0.2181            0.00s


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.15, loss='ls', max_depth=3,
                          max_features=15, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.15, n_estimators=10,
                          n_iter_no_change=None, presort='auto',
                          random_state=21, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [22]:
logytest = clf.predict(X_test)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("/content/drive/My Drive/CE4032/dataset/gradient_boost_13_Nov.csv", index=False)

Test shape OK.


In [32]:
print('training a random forest regressor ...')
# Initialize the famous Random Forest Regressor from scikit-learn
clf = RandomForestRegressor(n_estimators=10, max_depth = 3, min_weight_fraction_leaf= 0.15, max_features = 15, n_jobs=-1, random_state=21,verbose = 1)
clf.fit(X, y)

training a random forest regressor ...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   52.3s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features=15, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.15, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=21, verbose=1,
                      warm_start=False)

In [33]:
logytest = clf.predict(X_test)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("/content/drive/My Drive/CE4032/dataset/random_forest_13_Nov.csv", index=False)

Test shape OK.


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
